# Preliminaries
(Installations, imports, definitions. Reading this is not necessary.)

In [122]:
!pip install z3-solver

In [123]:
import sympy as sp
from IPython.display import display


x, y, u = sp.symbols('x y u')
gamma, gamma_dot, a = sp.symbols('\\gamma \dot{\\gamma} a')
alpha, beta, delta, e, p, q  = sp.symbols('\\alpha \\beta \\delta e p q')


def lie(field, F):
    vars = sorted(list(filter(lambda s: 'gamma' in s.name, F.free_symbols)),
                  key=lambda s: s.name)
    ys, xs = vars[:len(vars)//2], vars[len(vars)//2:]
    #display(xs)
    #display(ys)
    vars = [val for pair in zip(xs, ys) for val in pair]
    # print(vars)
    return (sp.Matrix([F]).jacobian(vars) @ field)

def fc_from_VGH(V, G, H, a=sp.Matrix([a])):
    f = sp.Rational(1, 4) * G @ lie(G, V).T - H
    c = lie(H, V)[0] + (a.T @ a)[0]
    return f, c.simplify().collect([y])

def peq(var, rhss):
    """pretty prints expresions of the kind $A = F(X, Y, Z)$"""
    if not isinstance(var, list):
        var = var.split()
    var_symbs = sp.symbols(" ".join(var))
    if len(var) == 1:
        var_symbs = [var_symbs]
        rhss = [rhss]
    for var_symb, rhs in zip(var_symbs, rhss):
        with sp.evaluate(False):
            display(sp.Eq(var_symb, rhs))

# assumes we're dealing with "concatenated cranks"
def print_problem(V, f, g, c, a=sp.Matrix([a])):
    peq('V', V)
    if not len(f) == 2:
        peq(" ".join([f'\\frac{{d}}{{dt}}{{\\gamma}}_{{{i}}} \\frac{{d}}{{dt}}\\dot{{\\gamma}}_{{{i}}}' for i in range(len(f) // 2)]), f + g @ a)
    else:
        peq("\\frac{d}{dt}\\gamma \\frac{d}{dt}\\dot{\\gamma}", f + g @ a)
    peq('c', c)

import sympy as sp
import z3
from typing import List, Dict, Tuple, Union, Optional

def sympy_to_z3(expr, variables_map: Dict[str, z3.ArithRef] = None) -> z3.ExprRef:
    """
    Convert a sympy expression to a Z3 expression.

    Args:
        expr: A sympy expression
        variables_map: A dictionary mapping sympy variable names to Z3 variables

    Returns:
        The equivalent Z3 expression
    """
    if variables_map is None:
        variables_map = {}

    # Handle constants
    if expr.is_number:
        return z3.RealVal(float(expr))

    # Handle symbols (variables)
    if expr.is_Symbol:
        var_name = expr.name
        if var_name not in variables_map:
            variables_map[var_name] = z3.Real(var_name.replace('\\', ''))
        return variables_map[var_name]

    # Handle addition
    if expr.is_Add:
        result = z3.RealVal(0)
        for arg in expr.args:
            result = result + sympy_to_z3(arg, variables_map)
        return result

    # Handle multiplication
    if expr.is_Mul:
        result = z3.RealVal(1)
        for arg in expr.args:
            result = result * sympy_to_z3(arg, variables_map)
        return result

    # Handle power
    if expr.is_Pow:
        base = sympy_to_z3(expr.base, variables_map)
        exp = expr.exp

        # Only handle integer exponents for simplicity
        if exp.is_Integer:
            exp_val = int(exp)
            return base ** exp_val
        elif isinstance(exp, sp.Rational):
            return z3.Sqrt(base)
    # if isinstance(expr, sp.cos):
    #     print("*cos*")
    #     return z3.cos(sympy_to_z3(expr.args[0], variables_map))

    # Handle subtraction (will be converted to addition with negative coefficient)

    # Handle division
    if isinstance(expr, sp.Mul) and any(arg.is_Pow and arg.exp.is_negative for arg in expr.args):
        # Extract the numerator and denominator
        num, den = sp.fraction(expr)
        return sympy_to_z3(num, variables_map) / sympy_to_z3(den, variables_map)

    # Add more operations as needed

    raise ValueError(f"Unsupported expression type: {type(expr)}")

def sympy_relation_to_z3(relation, variables_map: Dict[str, z3.ArithRef] = None) -> z3.BoolRef:
    """
    Convert a sympy relation to a Z3 constraint.

    Args:
        relation: A sympy relation (>, >=, =, <, <=, !=)
        variables_map: A dictionary mapping sympy variable names to Z3 variables

    Returns:
        The equivalent Z3 constraint
    """
    if variables_map is None:
        variables_map = {}

    # Check if relation is an actual sympy relational object
    if not isinstance(relation, sp.Rel):
        raise TypeError(f"Expected a sympy relational object, got {type(relation)}. Make sure you're using sympy relations and not Python's built-in comparisons.")

    # Get the left and right hand sides
    lhs, rhs = relation.lhs, relation.rhs

    # Convert to Z3 expressions
    z3_lhs = sympy_to_z3(lhs, variables_map)
    z3_rhs = sympy_to_z3(rhs, variables_map)

    # Create the appropriate Z3 constraint based on the relation type
    if isinstance(relation, sp.Gt):  # >
        return z3_lhs > z3_rhs
    elif isinstance(relation, sp.Ge):  # >=
        return z3_lhs >= z3_rhs
    elif isinstance(relation, sp.Eq):  # =
        return z3_lhs == z3_rhs
    elif isinstance(relation, sp.Lt):  # <
        return z3_lhs < z3_rhs
    elif isinstance(relation, sp.Le):  # <=
        return z3_lhs <= z3_rhs
    elif isinstance(relation, sp.Ne):  # !=
        return z3_lhs != z3_rhs
    else:
        raise ValueError(f"Unsupported relation type: {type(relation)}")

def ensure_sympy_relation(rel):
    """
    Ensure that the input is a sympy relational object.
    If it's already a sympy relation, return it.
    If it's a boolean, raise an error with a helpful message.

    Args:
        rel: A potential sympy relation

    Returns:
        The input if it's a sympy relation

    Raises:
        TypeError: If the input is not a sympy relation
    """
    if isinstance(rel, sp.Rel):
        return rel
    elif isinstance(rel, bool):
        raise TypeError(
            "Received a Python boolean instead of a sympy relation. "
            "This often happens when using '==' with sympy symbols. "
            "Use sp.Eq(x, y) instead of x == y to ensure you get a sympy relational object."
        )
    else:
        raise TypeError(f"Expected a sympy relational object, got {type(rel)}.")

def is_satisfiable(relations: List[sp.Rel], return_counterex=False) -> Tuple[bool, Optional[Dict[str, float]]]:
    """
    Check if a list of sympy relations are simultaneously satisfiable.

    Args:
        relations: A list of sympy relations

    Returns:
        A tuple containing:
            - A boolean indicating if the relations are satisfiable
            - If satisfiable, a dictionary mapping variable names to values
              that satisfy all relations. If not satisfiable, None.
    """
    # Create Z3 solver
    solver = z3.Solver()

    # Map to keep track of variables
    variables_map = {}

    # Convert each relation to Z3 and add to solver
    for relation in relations:
        # Ensure we have a sympy relation
        relation = ensure_sympy_relation(relation)
        z3_constraint = sympy_relation_to_z3(relation, variables_map)
        solver.add(z3_constraint)

    # Check satisfiability
    result = solver.check()
    # print(result)
    if result == z3.sat:
        # Get the model (a satisfying assignment)
        model = solver.model()

        # Convert model to a Python dictionary
        solution = {}
        for var_name, z3_var in variables_map.items():
            if model[z3_var] is not None:
                solution[var_name] = float(model[z3_var].as_decimal(10).replace('?', ''))

        return (True, solution) if return_counterex else True
    else:
        return (False, None) if return_counterex else False


def not_(relations):
    return list(map(lambda s: ~s, relations))


def remove_control(expr): ## assuming SISO
    return expr.subs(u, 0)

def is_pd(F, verbose=False, return_counterex=False):
    F = remove_control(F)
    zero_at_origin = F.subs({key : 0 for key in F.free_symbols}).simplify() == 0
    if not zero_at_origin:
        if verbose:
            print(F, "is not zero at the origin.")
        return (False, None) if return_counterex else False
    v = sp.Matrix(list(F.free_symbols))
    if not v:
        print(F, "has non-positive values everywhere.")
        return (False, None) if return_counterex else False
    has_nonpos_val, counterex = is_satisfiable([(v.T @ v)[0] > 0, F <= 0],
                                                      return_counterex=True)
    if has_nonpos_val and verbose:
        print(F, "has a non-positive value at", counterex)

    return (not has_nonpos_val, counterex) if return_counterex else not has_nonpos_val



def is_psd(F, verbose=False, return_counterex=False):
    F = remove_control(F)
    zero_at_origin = F.subs({key : 0 for key in F.free_symbols}).simplify() == 0
    if not zero_at_origin:
        if verbose:
            print(F, "is not zero at the origin.")
        return (False, None) if return_counterex else False
    has_neg_val, counterex = is_satisfiable([F < 0],
                                            return_counterex=True)
    if has_neg_val and verbose:
        print(F, "has a negative value at", counterex)

    return (not has_neg_val, counterex) if return_counterex else not has_neg_val


def check_pd(V, c):
    V_, c_ = sp.symbols(['V', 'c'])
    res = {}
    for symb, val in zip([V_, c_], [V, c]):
        print("Checking positive-definitness of", symb, "...")
        pd, counterex = is_pd(val, verbose=True, return_counterex=True)
        if pd:
            print("Confirmed.", symb, "is positive-definite.\n")
        else:
            print("Refuted.", symb, "is NOT positive-definite.\n")
        res[str(symb)] = [pd, counterex]
    return res


def pd_feasibility(V, c):
    c = remove_control(c)
    V_z3  = sympy_to_z3(V)
    c_z3  = sympy_to_z3(c)
    return z3.solve([z3.ForAll([z3.Real(x.name), z3.Real(y.name)],
                               z3.Implies(z3.Real(x.name) ** 2 + z3.Real(y.name) ** 2 > 0,
                                          z3.And(V_z3 > 0, c_z3 > 0)))])






# Crank Environments

## Single crank

### Symbols

In [124]:
M, R = sp.symbols('M R')
alpha, beta, delta = sp.symbols('\\alpha \\beta \\delta')

### Build the dynamics

First we define $G := \left({0} \atop {\cos(\gamma)}\right)$, so that the leading arm can only push the crank horizontally.

As a side effect we get the following term added to the dynamics: $\frac{1}{4}GG^T\nabla V$.

In order to cancel this term, we need to add it to $H$, since $\frac{d}{dt}\left({\gamma} \atop {\dot{\gamma}}\right) = \ldots - H$. But aside from it $H$ should also include $\left({0} \atop {M\gamma}\right)$ for gravity, $\left({0} \atop {R\dot{\gamma}}\right)$ for friction and $\left({-\dot{\gamma}} \atop {0}\right)$ to make sure $\frac{d}{dt}\gamma = \dot{\gamma}$.




In [125]:

V = alpha * gamma ** 2 + 2 * beta * gamma * gamma_dot +  delta * gamma_dot ** 2
G = sp.Matrix([0, sp.cos(gamma)])
H = sp.Matrix([-gamma_dot,  M*gamma + R * gamma_dot + (sp.Rational(1, 4) * G @ G.T @ sp.Matrix([V]).jacobian([gamma, gamma_dot]).T)[1]])

display(sp.Matrix([V]).jacobian([gamma, gamma_dot]).T)

peq('H', H)

f, c = fc_from_VGH(V, G, H)

print_problem(V, f, G, c)

Matrix([
[2*\alpha*\gamma + 2*\beta*\dot{\gamma}],
[2*\beta*\gamma + 2*\delta*\dot{\gamma}]])

Eq(H, Matrix([
[                                                                        -\dot{\gamma}],
[M*\gamma + R*\dot{\gamma} + (2*\beta*\gamma + 2*\delta*\dot{\gamma})*cos(\gamma)**2/4]]))

Eq(V, \alpha*\gamma**2 + 2*\beta*\dot{\gamma}*\gamma + \delta*\dot{\gamma}**2)

Eq(\frac{d}{dt}\gamma}, \dot{\gamma})

Eq(\frac{d}{dt}\dot{\gamma}, -M*\gamma - R*\dot{\gamma} + a*cos(\gamma))

Eq(c, -2*\dot{\gamma}*(\alpha*\gamma + \beta*\dot{\gamma}) + a**2 + (\beta*\gamma + \delta*\dot{\gamma})*(2*M*\gamma + 2*R*\dot{\gamma} + (\beta*\gamma + \delta*\dot{\gamma})*cos(\gamma)**2))

### Separate quadratic terms and derive p.d. criteria

Inspecting the cost reveals that it can be represented as $s^TQs + (q^Ts)^2\cos^2(\gamma)$ for some matrix $Q$ and some vector $q$, where $s = \left({\gamma} \atop {\dot \gamma}\right)$.

This means that analysis of positive definiteness of $c$ reduces to the analysis of positive-definiteness of $Q$.

In [126]:
c_stripped = c.subs(a, 0).subs(sp.cos(gamma), 0).expand().collect([gamma, gamma_dot])
peq("c_{stripped}", c_stripped)

m11 = c_stripped.coeff(gamma, 2)
m12 = c_stripped.coeff(gamma, 1).diff(gamma_dot) / 2
m21 = m12
m22 = c_stripped.coeff(gamma, 0).diff(gamma_dot).diff(gamma_dot) / 2

criteria = [m11 > 0, m11 * m22 - m21 * m12 > 0,
            alpha > 0, alpha * delta - beta ** 2 > 0]

display(criteria)



Eq(c_{stripped}, 2*M*\beta*\gamma**2 + \dot{\gamma}**2*(2*R*\delta - 2*\beta) + \gamma*(2*M*\delta*\dot{\gamma} + 2*R*\beta*\dot{\gamma} - 2*\alpha*\dot{\gamma}))

[2*M*\beta > 0,
 2*M*\beta*(2*R*\delta - 2*\beta) - (M*\delta + R*\beta - \alpha)**2 > 0,
 \alpha > 0,
 \alpha*\delta - \beta**2 > 0]

### Derive alternative criteria that are convenient for sampling

In [127]:
big_rel = (m11 * m22 - m21 * m12).expand().collect(delta)
C = big_rel.coeff(delta, 0)
B = big_rel.coeff(delta, 1)
A = big_rel.coeff(delta, 2)

D = (B**2 - 4 * A * C).simplify()

peq('A', A)
peq('B', B)
peq('C', C)
peq('D', D)

D_root = sp.symbols('D_root')


better_criteria = [M > 0, R > 0, beta > 0, alpha > M * beta / R,
                   delta > (-B + D_root) / (2 * A),
                   delta < (-B - D_root) / (2 * A),
                   sp.Eq(D_root * D_root, D)]
display(better_criteria)



Eq(A, -M**2)

Eq(B, 2*M*R*\beta + 2*M*\alpha)

Eq(C, -4*M*\beta**2 - R**2*\beta**2 + 2*R*\alpha*\beta - \alpha**2)

Eq(D, 16*M**2*\beta*(-M*\beta + R*\alpha))

[M > 0,
 R > 0,
 \beta > 0,
 \alpha > M*\beta/R,
 \delta > -(D_root - 2*M*R*\beta - 2*M*\alpha)/(2*M**2),
 \delta < -(-D_root - 2*M*R*\beta - 2*M*\alpha)/(2*M**2),
 Eq(D_root**2, 16*M**2*\beta*(-M*\beta + R*\alpha))]

### Formally verify that the latter criteria were derived correctly

In [128]:
## formal testing

better_criteria_are_satisfied = z3.And(*list(map(sympy_relation_to_z3, better_criteria)))
but_original_criteria_are_not_satisfied = z3.Not(z3.And(*list(map(sympy_relation_to_z3, criteria))))
print("Improved criteria are satisfiable:", is_satisfiable(better_criteria))
print("It is possible for the original criteria to be unsatisfied:", is_satisfiable(not_(criteria)))
print("It is possible that improved criteria are satisfied while original criteria are not:",
      is_satisfiable(criteria + not_(better_criteria)))

Improved criteria are satisfiable: True
It is possible for the original criteria to be unsatisfied: True
It is possible that improved criteria are satisfied while original criteria are not: False


### Test

In [129]:
## empirical testing

import numpy as np
from tqdm import tqdm

bottom_d = sp.lambdify([alpha, beta, M, R], (-B + sp.sqrt(D)) / (2 * A))
length_d = sp.lambdify([alpha, beta, M, R], -sp.sqrt(D) / A)

def generate_abdMR(size, range_=10):
    res = np.random.random(size=size * 5).reshape(size, 5) * range_
    for i in range(size):
        _, beta, _, M, R = res[i]

        res[i, 0] += M * beta / R,
        alpha = res[i, 0]

        res[i, 2] *= length_d(alpha, beta, M, R) / range_
        res[i, 2] += bottom_d(alpha, beta, M, R)
    return res



num_tests = 100
params = generate_abdMR(num_tests)

count_both_pd = 0
for param in tqdm(params):
    substitution = list(zip([alpha, beta, delta, M, R], param))
    c_ = c_stripped.subs(substitution)
    V_ = V.subs(substitution)
    res =  is_pd(c_) and is_pd(V_)
    count_both_pd += res
    if not res:
        print(is_pd(c_, return_counterex=True)[1],
              is_pd(V_, return_counterex=True)[1])
        print(V_, c_, sep=' | ')

print(f"\nRandom tests passed: {count_both_pd}/{num_tests}")

<ipython-input-129-ce70e2edd1d6>:14: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  res[i, 0] += M * beta / R,
100%|██████████| 100/100 [00:03<00:00, 25.93it/s]


Random tests passed: 100/100




## Integrating two cranks

Example for two cranks and concrete parameters




### Symbols

In [130]:
M, R = 1, 1 # sp.symbols('M R')
alpha, beta, delta = 3, 1, 3 # sp.symbols('a b d')
gamma_0, gamma_dot_0, gamma_1, gamma_dot_1 = sp.symbols('\gamma_0 \dot{\gamma}_0 \gamma_1 \dot{\gamma}_1')

a_mult = sp.Matrix(sp.symbols('a_0 a_1'))

### Define subsystems



In [131]:
V_n = alpha * gamma ** 2 + 2 * beta * gamma * gamma_dot +  delta * gamma_dot ** 2
G_n = sp.Matrix([0, sp.cos(gamma)])
H_n = sp.Matrix([-gamma_dot,  M*gamma + R * gamma_dot + (sp.Rational(1, 4) * G_n @ G_n.T @ sp.Matrix([V_n]).jacobian([gamma, gamma_dot]).T)[1]])


subs0 = [(gamma, gamma_0), (gamma_dot, gamma_dot_0)]
subs1 = [(gamma, gamma_1), (gamma_dot, gamma_dot_1)]





### Compose subsystems

In [132]:
V = V_n.subs(subs0) + V_n.subs(subs1)
G = sp.diag(G_n.subs(subs0), G_n.subs(subs1))
H = sp.Matrix.vstack(H_n.subs(subs0), H_n.subs(subs1))


### Add a spring

In [133]:
H[1] += 0.2 * (gamma_0 - gamma_1)
H[3] += 0.2 * (gamma_1 - gamma_0)

peq('H', H)

Eq(H, Matrix([
[                                                                                  -\dot{\gamma}_0],
[\dot{\gamma}_0 + 1.2*\gamma_0 - 0.2*\gamma_1 + (6*\dot{\gamma}_0 + 2*\gamma_0)*cos(\gamma_0)**2/4],
[                                                                                  -\dot{\gamma}_1],
[\dot{\gamma}_1 - 0.2*\gamma_0 + 1.2*\gamma_1 + (6*\dot{\gamma}_1 + 2*\gamma_1)*cos(\gamma_1)**2/4]]))

### Construct the dynamics and stage costs

In [134]:
f, c = fc_from_VGH(V, G, H)

print_problem(V, f, G, c, a=a_mult)





Eq(V, 3*\dot{\gamma}_0**2 + 2*\dot{\gamma}_0*\gamma_0 + 3*\dot{\gamma}_1**2 + 2*\dot{\gamma}_1*\gamma_1 + 3*\gamma_0**2 + 3*\gamma_1**2)

Eq(\frac{d}{dt}{\gamma}_{0}, \dot{\gamma}_0)

Eq(\frac{d}{dt}\dot{\gamma}_{0}, -\dot{\gamma}_0 - 1.2*\gamma_0 + 0.2*\gamma_1 + a_0*cos(\gamma_0))

Eq(\frac{d}{dt}{\gamma}_{1}, \dot{\gamma}_1)

Eq(\frac{d}{dt}\dot{\gamma}_{1}, -\dot{\gamma}_1 + 0.2*\gamma_0 - 1.2*\gamma_1 + a_1*cos(\gamma_1))

Eq(c, -2*\dot{\gamma}_0*(\dot{\gamma}_0 + 3*\gamma_0) - 2*\dot{\gamma}_1*(\dot{\gamma}_1 + 3*\gamma_1) + a**2 + (3*\dot{\gamma}_0 + \gamma_0)*(4*\dot{\gamma}_0 + 4.8*\gamma_0 - 0.8*\gamma_1 + 2*(3*\dot{\gamma}_0 + \gamma_0)*cos(\gamma_0)**2)/2 + (3*\dot{\gamma}_1 + \gamma_1)*(4*\dot{\gamma}_1 - 0.8*\gamma_0 + 4.8*\gamma_1 + 2*(3*\dot{\gamma}_1 + \gamma_1)*cos(\gamma_1)**2)/2)

### Verify correctness

In [135]:
c_stripped = c.subs(a, 0).subs(sp.cos(gamma_0), 0).subs(sp.cos(gamma_1), 0).expand().collect([gamma, gamma_dot])
peq("c_{stripped}", c_stripped)

is_pd(c_stripped)

Eq(c_{stripped}, 4*\dot{\gamma}_0**2 + 3.2*\dot{\gamma}_0*\gamma_0 - 1.2*\dot{\gamma}_0*\gamma_1 + 4*\dot{\gamma}_1**2 - 1.2*\dot{\gamma}_1*\gamma_0 + 3.2*\dot{\gamma}_1*\gamma_1 + 2.4*\gamma_0**2 - 0.8*\gamma_0*\gamma_1 + 2.4*\gamma_1**2)

True

## Integrating $N$ cranks



### Symbols

In [136]:
N = 4

def symb_array(names, count=N):
    res = []
    for name in names.split():
        res.append(sp.symbols(" ".join([name + "_" + str(i) for i in range(count)])))
    return res

alphas, betas, deltas, Ms, Rs, Ks = symb_array('\\alpha \\beta \\delta M R K')
Ks = Ks[:-1]
gammas, gamma_dots, as_ = symb_array('\\gamma \dot{\\gamma} a')

state = [s for gamma_, gamma_dot_ in zip(gammas, gamma_dots) for s in (gamma_, gamma_dot_)]

M, R = sp.symbols('M R')
alpha, beta, delta = sp.symbols('\alpha \beta \delta')
K = sp.symbols('K')

### Define and compose subsystems

It can be inferred that a system of independent cranks can be generated by

$$V = \sum_{i = 1}^{N}V_i,$$
$$G = \begin{pmatrix}G_1 & 0 & 0 & \ldots & 0\\
0 & G_2 & 0 &\ldots & 0\\
0 & 0 & G_3 &\ldots & 0\\
\ldots & \ldots & \ldots & \ldots & \ldots\\
0 & 0 & 0 & \ldots & G_N,
\end{pmatrix}$$
$$H = \begin{pmatrix}H_1\\ \ldots \\ H_N\end{pmatrix},$$




In [137]:


V_n = alpha * gamma ** 2 + 2 * beta * gamma * gamma_dot +  delta * gamma_dot ** 2
G_n = sp.Matrix([0, sp.cos(gamma)])
H_n = sp.Matrix([-gamma_dot,  M*gamma + R * gamma_dot + (sp.Rational(1, 4) * G_n @ G_n.T @ sp.Matrix([V_n]).jacobian([gamma, gamma_dot]).T)[1]])

V = \
  sum([V_n.subs([(gamma, gammas[i]), (gamma_dot, gamma_dots[i]), (alpha, alphas[i]), (beta, betas[i]), (delta, deltas[i])])
       for i in range(N)])
G = sp.diag(*[G_n.subs(gamma, gammas[i]) for i in range(N)])
H = sp.Matrix.vstack(*[H_n.subs([(gamma, gammas[i]), (gamma_dot, gamma_dots[i]), (alpha, alphas[i]),
                                 (beta, betas[i]), (delta, deltas[i]), (M, Ms[i]), (R, Rs[i])]) \
                     for i in range(N)])


### Add springs

These independent subsystems can now be linked by setting
$$H_{\text{new}} := \begin{pmatrix}H_1 + K_1(\gamma_1 - \gamma_2)\\ \ldots \\
H_{N - 1} + K_{N - 2}(\gamma_{N - 1} - \gamma_{N - 2}) + K_{N - 1}(\gamma_{N - 1} - \gamma_{N})\\
H_N + K_{N - 1}(\gamma_{N} - \gamma_{N - 1})\end{pmatrix},$$

In [138]:
pad = sp.symbols('pad')
for i in range(N):
    padded_gammas = list(gammas) + [pad]
    padded_Ks = list(Ks) + [pad]
    H[2 * i + 1] += padded_Ks[i - 1] * (padded_gammas[i] - padded_gammas[i - 1]) # enacted by previous crank
    H[2 * i + 1] += padded_Ks[i] * (padded_gammas[i] - padded_gammas[i + 1]) # enacted by next crank
H = H.subs(pad, 0)

peq('H', H)

Eq(H, Matrix([
[                                                                                                                                                 -\dot{\gamma}_0],
[                             K_0*(\gamma_0 - \gamma_1) + M_0*\gamma_0 + R_0*\dot{\gamma}_0 + (2*\beta_0*\gamma_0 + 2*\delta_0*\dot{\gamma}_0)*cos(\gamma_0)**2/4],
[                                                                                                                                                 -\dot{\gamma}_1],
[K_0*(-\gamma_0 + \gamma_1) + K_1*(\gamma_1 - \gamma_2) + M_1*\gamma_1 + R_1*\dot{\gamma}_1 + (2*\beta_1*\gamma_1 + 2*\delta_1*\dot{\gamma}_1)*cos(\gamma_1)**2/4],
[                                                                                                                                                 -\dot{\gamma}_2],
[K_1*(-\gamma_1 + \gamma_2) + K_2*(\gamma_2 - \gamma_3) + M_2*\gamma_2 + R_2*\dot{\gamma}_2 + (2*\beta_2*\gamma_2 + 2*\delta_2*\dot{\gamma}_2)*cos(\gamma_2)**2/4],
[

### Construct the dynamics and costs

In [139]:

f, c = fc_from_VGH(V, G, H, a=sp.Matrix(as_))

print_problem(V, f, G, c, a=sp.Matrix(as_))


Eq(V, \alpha_0*\gamma_0**2 + \alpha_1*\gamma_1**2 + \alpha_2*\gamma_2**2 + \alpha_3*\gamma_3**2 + 2*\beta_0*\dot{\gamma}_0*\gamma_0 + 2*\beta_1*\dot{\gamma}_1*\gamma_1 + 2*\beta_2*\dot{\gamma}_2*\gamma_2 + 2*\beta_3*\dot{\gamma}_3*\gamma_3 + \delta_0*\dot{\gamma}_0**2 + \delta_1*\dot{\gamma}_1**2 + \delta_2*\dot{\gamma}_2**2 + \delta_3*\dot{\gamma}_3**2)

Eq(\frac{d}{dt}{\gamma}_{0}, \dot{\gamma}_0)

Eq(\frac{d}{dt}\dot{\gamma}_{0}, -K_0*(\gamma_0 - \gamma_1) - M_0*\gamma_0 - R_0*\dot{\gamma}_0 + a_0*cos(\gamma_0))

Eq(\frac{d}{dt}{\gamma}_{1}, \dot{\gamma}_1)

Eq(\frac{d}{dt}\dot{\gamma}_{1}, -K_0*(-\gamma_0 + \gamma_1) - K_1*(\gamma_1 - \gamma_2) - M_1*\gamma_1 - R_1*\dot{\gamma}_1 + a_1*cos(\gamma_1))

Eq(\frac{d}{dt}{\gamma}_{2}, \dot{\gamma}_2)

Eq(\frac{d}{dt}\dot{\gamma}_{2}, -K_1*(-\gamma_1 + \gamma_2) - K_2*(\gamma_2 - \gamma_3) - M_2*\gamma_2 - R_2*\dot{\gamma}_2 + a_2*cos(\gamma_2))

Eq(\frac{d}{dt}{\gamma}_{3}, \dot{\gamma}_3)

Eq(\frac{d}{dt}\dot{\gamma}_{3}, -K_2*(-\gamma_2 + \gamma_3) - M_3*\gamma_3 - R_3*\dot{\gamma}_3 + a_3*cos(\gamma_3))

Eq(c, -2*\dot{\gamma}_0*(\alpha_0*\gamma_0 + \beta_0*\dot{\gamma}_0) - 2*\dot{\gamma}_1*(\alpha_1*\gamma_1 + \beta_1*\dot{\gamma}_1) - 2*\dot{\gamma}_2*(\alpha_2*\gamma_2 + \beta_2*\dot{\gamma}_2) - 2*\dot{\gamma}_3*(\alpha_3*\gamma_3 + \beta_3*\dot{\gamma}_3) + a_0**2 + a_1**2 + a_2**2 + a_3**2 + (\beta_0*\gamma_0 + \delta_0*\dot{\gamma}_0)*(2*K_0*(\gamma_0 - \gamma_1) + 2*M_0*\gamma_0 + 2*R_0*\dot{\gamma}_0 + (\beta_0*\gamma_0 + \delta_0*\dot{\gamma}_0)*cos(\gamma_0)**2) + (\beta_1*\gamma_1 + \delta_1*\dot{\gamma}_1)*(-2*K_0*(\gamma_0 - \gamma_1) + 2*K_1*(\gamma_1 - \gamma_2) + 2*M_1*\gamma_1 + 2*R_1*\dot{\gamma}_1 + (\beta_1*\gamma_1 + \delta_1*\dot{\gamma}_1)*cos(\gamma_1)**2) + (\beta_2*\gamma_2 + \delta_2*\dot{\gamma}_2)*(-2*K_1*(\gamma_1 - \gamma_2) + 2*K_2*(\gamma_2 - \gamma_3) + 2*M_2*\gamma_2 + 2*R_2*\dot{\gamma}_2 + (\beta_2*\gamma_2 + \delta_2*\dot{\gamma}_2)*cos(\gamma_2)**2) + (\beta_3*\gamma_3 + \delta_3*\dot{\gamma}_3)*(-2*K_2*(\gamma_2 - \gamma_3) + 2*M_3*\gamma_3 + 2*

### Separate quadratic terms and isolate the contribution of springs

In [140]:
c_stripped = c
for i in range(N):
    c_stripped = c_stripped.subs(as_[i], 0).subs(sp.cos(gammas[i]), 0)
Q = sp.Matrix([c_stripped]).jacobian(state).T.jacobian(state) / 2

Q_no_springs = Q.subs(list(zip(Ks, [0] * (N - 1))))
DeltaQ = Q - Q_no_springs


peq('c_{stripped|same|K}', c_stripped.subs(list(zip(Ks, [K] * (N - 1)))).expand().collect(K))

peq("\Delta{Q}", DeltaQ)

Eq(c_{stripped|same|K}, K*(2*\beta_0*\gamma_0**2 - 2*\beta_0*\gamma_0*\gamma_1 - 2*\beta_1*\gamma_0*\gamma_1 + 4*\beta_1*\gamma_1**2 - 2*\beta_1*\gamma_1*\gamma_2 - 2*\beta_2*\gamma_1*\gamma_2 + 4*\beta_2*\gamma_2**2 - 2*\beta_2*\gamma_2*\gamma_3 - 2*\beta_3*\gamma_2*\gamma_3 + 2*\beta_3*\gamma_3**2 + 2*\delta_0*\dot{\gamma}_0*\gamma_0 - 2*\delta_0*\dot{\gamma}_0*\gamma_1 - 2*\delta_1*\dot{\gamma}_1*\gamma_0 + 4*\delta_1*\dot{\gamma}_1*\gamma_1 - 2*\delta_1*\dot{\gamma}_1*\gamma_2 - 2*\delta_2*\dot{\gamma}_2*\gamma_1 + 4*\delta_2*\dot{\gamma}_2*\gamma_2 - 2*\delta_2*\dot{\gamma}_2*\gamma_3 - 2*\delta_3*\dot{\gamma}_3*\gamma_2 + 2*\delta_3*\dot{\gamma}_3*\gamma_3) + 2*M_0*\beta_0*\gamma_0**2 + 2*M_0*\delta_0*\dot{\gamma}_0*\gamma_0 + 2*M_1*\beta_1*\gamma_1**2 + 2*M_1*\delta_1*\dot{\gamma}_1*\gamma_1 + 2*M_2*\beta_2*\gamma_2**2 + 2*M_2*\delta_2*\dot{\gamma}_2*\gamma_2 + 2*M_3*\beta_3*\gamma_3**2 + 2*M_3*\delta_3*\dot{\gamma}_3*\gamma_3 + 2*R_0*\beta_0*\dot{\gamma}_0*\gamma_0 + 2*R_0*\del

Eq(\Delta{Q}, Matrix([
[  -2*M_0*\beta_0 + \beta_0*(2*K_0 + 2*M_0), -M_0*\delta_0 + \delta_0*(2*K_0 + 2*M_0)/2,                         -K_0*\beta_0 - K_0*\beta_1,                                      -K_0*\delta_1,                                                  0,                                                  0,                                          0,                                          0],
[-M_0*\delta_0 + \delta_0*(2*K_0 + 2*M_0)/2,                                          0,                                      -K_0*\delta_0,                                                  0,                                                  0,                                                  0,                                          0,                                          0],
[                -K_0*\beta_0 - K_0*\beta_1,                              -K_0*\delta_0,   -2*M_1*\beta_1 + \beta_1*(2*K_0 + 2*K_1 + 2*M_1), -M_1*\delta_1 + \delta_1*(2*K_0 + 2*K_1 + 2*M_1)/2,             

### Construct sampling map for elastic moduli

#### Useful bounds


It is known that a p.d. matrix $A$ stays p.d. after perturbation $\Delta A$ if
$$ \lVert \Delta A \rVert_2\lVert  A^{-1} \rVert_2 < 1.$$

Trivially, this is implied by
$$\lVert \Delta A \rVert_F^2 < \frac{\lvert A \rvert^2}{\lVert{\text{adj }A\rVert_F^2}}$$

Additionally if one assumes diagonal structure of $A$, we obtain:
$$\lVert \Delta A \rVert_F^2 < \frac{\prod_i \lvert A_i \rvert^2}{\sum_j\lVert{\text{adj }A_j\rVert_F^2 }\prod_{i \neq j}\lvert A_i \rvert^2}$$

If one further assumes that $A$ is $2\times 2$ and symmetric, we get
$$\lVert \Delta A \rVert_F^2 < \frac{\prod_i (a_{i11}a_{i22} - a_{i12}^2)^2}{\sum_j (a_{j11}^2 + 2a_{j12}^2 + a_{j22}^2)\prod_{i \neq j}(a_{i11}a_{i22} - a_{i12}^2)^2}$$

or alternatively:
$$\frac{1}{\lVert \Delta A \rVert_F^2} > \sum_j \frac{a_{j11}^2 + 2a_{j12}^2 + a_{j22}^2}{(a_{j11}a_{j22} - a_{j12}^2)^2}$$


In [141]:
DeltaQ_vectorized = DeltaQ.reshape(len(DeltaQ), 1)
DeltaQ_F2 = DeltaQ_vectorized.dot(DeltaQ_vectorized)
peq("||\Delta{Q}||_F^2", DeltaQ_F2.expand().collect(Ks[0]))
P = sp.Matrix([DeltaQ_F2]).jacobian(Ks).T.jacobian(Ks) / 2
peq("P", P)
L, D = ((P + P.T)/2).LDLdecomposition(hermitian=False)
peq("LDL", (L, D))


print("The linear part is zero:")
peq("F", sp.Matrix([DeltaQ_F2]).jacobian(Ks).subs(list(zip(Ks, [0] * (N - 1)))))


## Let's compute the inverse
peq("Q_{initial}", Q_no_springs)
Q_no_springs_inv = sp.diag(*[Q_no_springs[2 * i: 2 * i + 2, 2 * i: 2 * i + 2].inv() for i in range(N)])
peq("Q_{initial|inverse}", Q_no_springs_inv)

Q_no_springs_inv_vec = Q_no_springs_inv.reshape(len(DeltaQ), 1)


rhs = 1 / Q_no_springs_inv_vec.dot(Q_no_springs_inv_vec)


# transform from unit ball that yields valid Ks
transform = L.T.inv() @ D.inv(method="LU").pow(sp.Rational(1, 2)) * sp.sqrt(rhs)



Eq(||\Delta{Q}||_F^2, K_0**2*(6*\beta_0**2 + 4*\beta_0*\beta_1 + 6*\beta_1**2 + 4*\delta_0**2 + 4*\delta_1**2) + K_0*(8*K_1*\beta_1**2 + 4*K_1*\delta_1**2) + 6*K_1**2*\beta_1**2 + 4*K_1**2*\beta_1*\beta_2 + 6*K_1**2*\beta_2**2 + 4*K_1**2*\delta_1**2 + 4*K_1**2*\delta_2**2 + 8*K_1*K_2*\beta_2**2 + 4*K_1*K_2*\delta_2**2 + 6*K_2**2*\beta_2**2 + 4*K_2**2*\beta_2*\beta_3 + 6*K_2**2*\beta_3**2 + 4*K_2**2*\delta_2**2 + 4*K_2**2*\delta_3**2)

Eq(P, Matrix([
[4*\beta_0**2 + 4*\beta_1**2 + 4*\delta_0**2 + 4*\delta_1**2 + (-2*\beta_0 - 2*\beta_1)*(-\beta_0 - \beta_1),                                                                                4*\beta_1**2 + 2*\delta_1**2,                                                                                                           0],
[                                                                               4*\beta_1**2 + 2*\delta_1**2, 4*\beta_1**2 + 4*\beta_2**2 + 4*\delta_1**2 + 4*\delta_2**2 + (-2*\beta_1 - 2*\beta_2)*(-\beta_1 - \beta_2),                                                                                4*\beta_2**2 + 2*\delta_2**2],
[                                                                                                          0,                                                                                4*\beta_2**2 + 2*\delta_2**2, 4*\beta_2**2 + 4*\beta_3**2 + 4*\delta_2**2 + 4*\delta_3**2 + (-2*\beta_2 - 2*\beta_3)*(-\beta_2 - \beta_3)]

Eq(LDL, (Matrix([
[                                                                                                                                           1,                                                                                                                                                                                                                                                                                              0, 0],
[(4*\beta_1**2 + 2*\delta_1**2)/(4*\beta_0**2 + 4*\beta_1**2 + 4*\delta_0**2 + 4*\delta_1**2 + (-2*\beta_0 - 2*\beta_1)*(-\beta_0 - \beta_1)),                                                                                                                                                                                                                                                                                              1, 0],
[                                                                                                               

The linear part is zero:


Eq(F, Matrix([[0, 0, 0]]))

Eq(Q_{initial}, Matrix([
[                        2*M_0*\beta_0, M_0*\delta_0 + R_0*\beta_0 - \alpha_0,                                     0,                                     0,                                     0,                                     0,                                     0,                                     0],
[M_0*\delta_0 + R_0*\beta_0 - \alpha_0,            2*R_0*\delta_0 - 2*\beta_0,                                     0,                                     0,                                     0,                                     0,                                     0,                                     0],
[                                    0,                                     0,                         2*M_1*\beta_1, M_1*\delta_1 + R_1*\beta_1 - \alpha_1,                                     0,                                     0,                                     0,                                     0],
[                                

Eq(Q_{initial|inverse}, Matrix([
[          (-2*R_0*\delta_0 + 2*\beta_0)/(M_0**2*\delta_0**2 - 2*M_0*R_0*\beta_0*\delta_0 - 2*M_0*\alpha_0*\delta_0 + 4*M_0*\beta_0**2 + R_0**2*\beta_0**2 - 2*R_0*\alpha_0*\beta_0 + \alpha_0**2), (M_0*\delta_0 + R_0*\beta_0 - \alpha_0)/(M_0**2*\delta_0**2 - 2*M_0*R_0*\beta_0*\delta_0 - 2*M_0*\alpha_0*\delta_0 + 4*M_0*\beta_0**2 + R_0**2*\beta_0**2 - 2*R_0*\alpha_0*\beta_0 + \alpha_0**2),                                                                                                                                                                                                 0,                                                                                                                                                                                                 0,                                                                                                                                                                                           

In [142]:
transform_matrix = sp.lambdify(alphas + betas + deltas + Ms + Rs, transform)

### Test

In [143]:
def generate_abdMR(size, range_=10):
    res = np.random.random(size=size * 5).reshape(size, 5) * range_
    for i in range(size):
        _, beta, _, M, R = res[i]

        res[i, 0] += M * beta / R,
        alpha = res[i, 0]

        res[i, 2] *= length_d(alpha, beta, M, R) / range_
        res[i, 2] += bottom_d(alpha, beta, M, R)
    return res


def sample_unit_ball(dim=N-1):
    while True:
        res = np.random.random(size=dim) * 2 - 1
        if res @ res < 1:
            return res



def gen_params(size, range_=30):
    samples = []
    for i in range(size):
        params = generate_abdMR(N, range_=range_)
        alphas_, betas_, deltas_, Ms_, Rs_ = params.T.tolist()
        transform_ = transform_matrix(*(alphas_ + betas_ + deltas_ + Ms_ + Rs_))
        p = sample_unit_ball()
        subs = dict(zip(alphas + betas + deltas + Ms + Rs, params.T.flatten().tolist()))
        subs.update(dict(zip(Ks, (transform_ @ p).tolist())))
        samples.append(subs)
    return samples


num_tests = 30
params = gen_params(num_tests)

count_both_pd = 0
for param in tqdm(params):
    print(param)
    c_ = c_stripped.subs(param)
    V_ = V.subs(param)
    res =  is_pd(c_) and is_pd(V_)
    count_both_pd += res
    if not res:
        print(is_pd(c_, return_counterex=True)[1],
              is_pd(V_, return_counterex=True)[1])
        print(V_, c_, sep=' | ')

print(f"\nRandom tests passed: {count_both_pd}/{num_tests}")






<ipython-input-143-4002124431bb>:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  res[i, 0] += M * beta / R,
  0%|          | 0/30 [00:00<?, ?it/s]

{\alpha_0: 34.277606694252704, \alpha_1: 20.880863702700008, \alpha_2: 28.077015689514266, \alpha_3: 16.364188947593284, \beta_0: 24.45956975050775, \beta_1: 0.6879623302624049, \beta_2: 1.3248585981845373, \beta_3: 21.20093652112663, \delta_0: 67.13259720850346, \delta_1: 2.096700436695644, \delta_2: 0.5061310249642229, \delta_3: 58.6745103085628, M_0: 12.739454016694852, M_1: 13.326979191805833, M_2: 22.736656990278163, M_3: 8.125452572331, R_0: 25.029921760781594, R_1: 5.031009046006153, R_2: 8.174735220513199, R_3: 22.4651209382709, K_0: -0.006882255151145188, K_1: -0.005391613179342192, K_2: 0.0019229835790989334}


  3%|▎         | 1/30 [00:00<00:07,  3.71it/s]

{\alpha_0: 38.4682545786093, \alpha_1: 33.91126339265964, \alpha_2: 46.45675852470047, \alpha_3: 25.497638027020756, \beta_0: 27.906646334835983, \beta_1: 27.71316952590236, \beta_2: 24.421920612329078, \beta_3: 20.855673123116933, \delta_0: 58.632158205288626, \delta_1: 84.45682959034502, \delta_2: 33.750540352202364, \delta_3: 37.35303464330934, M_0: 10.140594846733675, M_1: 10.524582568465394, M_2: 14.490263208067741, M_3: 7.928096886896078, R_0: 27.19782710609107, R_1: 29.84166575166868, R_2: 18.605913885213234, R_3: 9.817250658268922, K_0: -0.0011562200469330203, K_1: 0.005957003811186741, K_2: 0.010711112394275578}


  7%|▋         | 2/30 [00:00<00:07,  3.79it/s]

...

{\alpha_0: 34.710658362964345, \alpha_1: 28.984379049194665, \alpha_2: 27.76183624916264, \alpha_3: 27.437009868905943, \beta_0: 16.881805986924135, \beta_1: 5.673613326953786, \beta_2: 4.691644841968346, \beta_3: 14.23789234830454, \delta_0: 9.866448439628531, \delta_1: 11.156146551392334, \delta_2: 1.7628614343952393, \delta_3: 13.100021200179395, M_0: 18.408079818141378, M_1: 7.8145007729662, M_2: 25.87698190139883, M_3: 28.0296448650059, R_0: 12.97445582109412, R_1: 8.062177250457257, R_2: 8.114866930281382, R_3: 25.18156369556972, K_0: -0.0011140547834222181, K_1: 0.05072885403766256, K_2: 0.05686596652125608}


 97%|█████████▋| 29/30 [00:08<00:00,  3.75it/s]

{\alpha_0: 23.505809930734852, \alpha_1: 16.04578148982907, \alpha_2: 139.16974848545323, \alpha_3: 81.84754939761847, \beta_0: 19.19460965813764, \beta_1: 14.076239268070985, \beta_2: 14.658762056350675, \beta_3: 5.917869836156491, \delta_0: 46.707974583142224, \delta_1: 60.578022411768515, \delta_2: 9.020060891158597, \delta_3: 11.37724706200821, M_0: 10.294182314013877, M_1: 7.502536218355354, M_2: 23.45438218525349, M_3: 7.426255005806289, R_0: 29.58332696560804, R_1: 26.386702326103396, R_2: 2.8024743115617943, R_3: 0.6660865585602027, K_0: 0.0082614911582716, K_1: -0.004489990501297457, K_2: 0.0044498661097712245}


100%|██████████| 30/30 [00:08<00:00,  3.33it/s]


Random tests passed: 30/30
